# Capstone - Financial Access in the San Francisco Bay Area

by Julian Darby

# Introduction

Throughout the IBM Data Science Professional Certification, a focus has been on applying data science to solving tangible business problems. In this capstone project, I am required to utilise the Foursquare API, one of the leading sources of venue and location information, for that purpose.

As a current MBA student at Berkeley Haas, an area of particular interest to me has been expanding financial access, primarily through the development of financial technology (or 'fintech'). However for many Americans physical locations, such as bank branches, remain an important facet of their financial interactions, even in the heart of technology: the San Francisco Bay Area. Further, access to financial services remains uneven, and likely affected by demographic composition of communities. This provides an interesting opportunity to leverage Foursquare data, to inform stakeholders of the current state of physical financial access in the Bay Area. 

In this project, I have combined data from disparate sources to gather insights on the distribution and characteristics of financial locations. This information would be useful for the executives of a financial services company, for example to identify under-served communities within the Bay Area.

Coronavirus has provided a fascinating lens through which to view a project like this too. While, there has been a trend towards online banking as a result (American Banker, 2020) there is no certainty whether this will persist following the relaxation of stay-in-place restrictions. However, if consumers continue to trend towards using online services then it can be expected that some physical locations may need to close over the coming years. The insights gathered from this report could be useful in informing decisions as to which locations should be prioritised for continued operation.

# Data

The data for this project has primarily been sourced from the following locations:

1. sfgov.org
2. census.gov
3. foursquare.com

The website sfgov.org contains information on the ZIP codes contained within the Bay Area, as well the geometric data required for visualisation of these ZIP codes. The website census.gov contains the US Census data. Ideally, full Census data would be used to give the maximal richness of data possible. However, the decennial US Census does not have information available at the ZIP code level, so cannot be used for this purpose. An alternative, used in this project, is the American Community Survey, which has demographic information available at the ZIP Code Tabulation Area (ZCTA) level. These do not correspond perfectly to ZIP codes in every case, but cross-walks are available to help with checking the mapping from ZCTA to ZIP code.

Finally, Foursquare has been used as a source of location and venue data. As the focus of this project is on financial access, the API was used to search for venues that are primarily categorised as financial (ATM, Bank, Cheque Checking Service, Credit Union, Currency Exchange, Financial or Legal Office, or Insurance Office). This data is combined with the American Community Survey to draw insights on the state of financial access in the San Francisco Bay Area.

Please refer to the methodology section below for the application of these data sources.

# Methodology

## Importing Libraries and Setting Global Options

In [1]:
# installers for packages as required:
# !pip install sodapy
# !pip install geojson

import pandas as pd
from pandas.core.common import flatten
from sodapy import Socrata
import folium
import json
import geojson
from geojson import Feature, Point, FeatureCollection, MultiPolygon, Polygon
import numpy as np
import requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Defining the Geographic Boundaries of the Project

Collect geojson data for Bay Area ZIP codes from data.sf.gov

In [2]:
url_1 = "https://data.sfgov.org/api/geospatial/u5j3-svi6?method=export&format=GeoJSON"

geojson_data = json.loads(requests.get(url_1).text)

Collect Bay Area ZIP codes and city names from data.sf.gov

In [3]:
client = Socrata("data.sfgov.org", None) # unauthenticated client (only works with public data sets)
results = client.get("f9wk-m4qb", limit = 2000) # getting Bay Area ZIP codes

sfgov_df = pd.DataFrame.from_records(results) # convert to pandas dataframe
sfgov_df = sfgov_df.astype({'zip':'int64','area':'float64'}) # change data-type of zip to integer and area to float

Reorganize columns

In [4]:
ZIP_df = sfgov_df[['zip','po_name','area','the_geom']] # selecting three relevant columns
ZIP_df.columns = ['ZIP_code','city_name','area','geometry'] # rename columns
len(ZIP_df)

187

Collect ZIP to ZCTA cross-walk from usdmapper.org. US census data uses ZCTA (ZIP Code Tabulation Areas) which can differ from ZIP Codes. A cross-walk is a data set that helps to reconcile these two different methodologies.

In [5]:
url_2 = "https://www.udsmapper.org/docs/zip_to_zcta_2019.xlsx"

ZIP_to_ZCTA = pd.read_excel(url_2)

ZIP_to_ZCTA = ZIP_to_ZCTA[ZIP_to_ZCTA['ZCTA'] != 'No ZCTA'] # drop 'No ZCTA' data from dataframe
ZIP_to_ZCTA = ZIP_to_ZCTA.astype({'ZCTA':'int64'}) # change data-type of ZCTA to integer

Conduct search for mis-matched ZIPs/ZCTAs from within the cross-walk data set. Important for no poor mapping to occur to allow use of the American Community Survey data.

In [6]:
ZIP_match = 0 # initializing ZIP_match at zero
ill_defined = 0 # initializing ill_defined at zero
drop_ZIPs = [] # initalizing drop_ZIPs (i.e. ZIP Codes to drop)
dash_1 = '-' * 45 
table_format_1 = '{:<12}{:<12}{:<20s}'

print(dash_1)
print(table_format_1.format('ZIP Code','ZCTA','Join type'))
print(dash_1)

for ZIP in ZIP_df['ZIP_code']:
    if list(ZIP_to_ZCTA.loc[(ZIP_to_ZCTA['ZIP_CODE'] == ZIP),'ZCTA'])[0] != ZIP: # if ZIP Code not equal to ZTCA:
        ZCTA = list(ZIP_to_ZCTA.loc[(ZIP_to_ZCTA['ZIP_CODE'] == ZIP),'ZCTA'])[0]
        explanation = list(ZIP_to_ZCTA.loc[(ZIP_to_ZCTA['ZIP_CODE'] == ZIP),'zip_join_type'])[0]
        drop_ZIPs.append(ZIP)
        print(table_format_1.format(ZIP,ZCTA,explanation))
    
    elif list(ZIP_to_ZCTA.loc[(ZIP_to_ZCTA['ZIP_CODE'] == ZIP),'zip_join_type'])[0] == "Zip Matches ZCTA":
        # else if ZIP matches ZCTA:
        ZIP_match = ZIP_match + 1
        
    else:
        ill_defined = ill_defined + 1
        
print()
print("Matching ZIPs: %i" % ZIP_match) # matches being when ZIP Code = ZCTA
print("Ill-defined ZIPs: %i" % ill_defined) # ill-defined being anything missed by the for loop

---------------------------------------------
ZIP Code    ZCTA        Join type           
---------------------------------------------
94035       94043       Spatial join to ZCTA

Matching ZIPs: 186
Ill-defined ZIPs: 0


Drop mis-matched ZIPs from dataframe (in this instance, just ZIP code 94035 is mis-matched). Mis-matched in this context means that it is not a 1:1 mapping, rather than because of an error per se.

In [7]:
bay_ZIPs = ZIP_df.set_index('ZIP_code') # set index to allow drop function to operate

for ZIP in drop_ZIPs:
    bay_ZIPs = bay_ZIPs.drop(ZIP,axis = 0)
    
bay_ZIPs.reset_index(inplace = True) # reset index after drop
bay_ZIPs.head()

,ZIP_code,city_name,area,geometry
0,94558,NAPA,1.231326e+10,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
1,94533,FAIRFIELD,9.917861e+08,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
2,95620,DIXON,7.236950e+09,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
3,95476,SONOMA,3.001414e+09,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
4,94559,NAPA,1.194302e+09,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."


Choose the boundaries of the geographic area under consideration. This can either be the full Bay Area ZIP code region, or a subset.

In [8]:
ZIP_coordinates = bay_ZIPs['geometry']

all_coordinates = []

for i, coordinates in enumerate(ZIP_coordinates):
    all_coordinates.append(ZIP_coordinates[i]['coordinates'])
    
all_coordinates = list(flatten(all_coordinates)) # flattened list of coordinates in Bay Area geometry
all_latitudes = all_coordinates[1::2] # all Bay Area latitudes in geometry
all_longitudes = all_coordinates[0::2] # all Bay Area longitudes in geometry

search_range_custom = True # choose between searching full Bay Area, or user-defined range
custom_NE_corner = [38.232169,-121.929691] # user-defined NE corner
custom_SW_corner = [37.303010,-122.522385] # user-defind SW corner

if search_range_custom == True: # if True return user-defined boundaries, otherwise full Bay Area boundaries
    max_latitude = custom_NE_corner[0]
    min_latitude = custom_SW_corner[0]
    max_longitude = custom_NE_corner[1]
    min_longitude = custom_SW_corner[1]   

else: 
    max_latitude = max(all_latitudes)
    min_latitude = min(all_latitudes)
    max_longitude = max(all_longitudes)
    min_longitude = min(all_longitudes)

Summary of the coordinates of the corners of the geographic boundary

In [9]:
def accuracy(number):
    return round(number,4)

dash_2 = '-' * 45 
table_format_2 = '{:<15}{:<15}{:<15}'

print(dash_2)
print(table_format_2.format('Corner','Latitude','Longitude'))
print(dash_2)

print(table_format_2.format('NE',accuracy(max_latitude),accuracy(max_longitude)))
print(table_format_2.format('SE',accuracy(min_latitude),accuracy(max_longitude)))
print(table_format_2.format('SW',accuracy(min_latitude),accuracy(min_longitude)))
print(table_format_2.format('NW',accuracy(max_latitude),accuracy(min_longitude)))

---------------------------------------------
Corner         Latitude       Longitude      
---------------------------------------------
NE             38.2322        -121.9297      
SE             37.303         -121.9297      
SW             37.303         -122.5224      
NW             38.2322        -122.5224      


This geographic area is then sub-divided for visualisation later and to provide smaller search areas to the Foursquare API. The level of precision is set here.

In [10]:
precision = 21 # set sub-divisions of overall area to do venue search over

lat_linspace = np.linspace(min_latitude,max_latitude,precision) # sets interval latitudes
long_linspace = np.linspace(min_longitude,max_longitude,precision) # sets interval longitudes

print('there are %i sub-divisions in area' % (precision -1) ** 2) # gives total number of subdivisons

there are 400 sub-divisions in area


Next, check to see how many ZIP codes are completely contained within the search grid. Note: this will be the same as the total ZIP codes if not using the user-defined boundaries (because the boundaries will be a function of the ZIP codes themselves).

## Collecting Demographic Information from the American Community Survey

Now that the ZIP code data has been cleaned up, download American Community Survery information from data.census.gov and create a dataframe with entries corresponding to the ZIP codes of interest.

In [11]:
ACS_name = 'ACSDP5Y2018.DP05_data_with_overlays_2020-05-04T030452.csv' # name of downloaded file

ACS_df = pd.read_csv(ACS_name, header = 1, low_memory = False) # skip first line of csv

ACS_column_names = list(ACS_df.columns) # create list of ACS column names
ACS_column_names_data = ACS_column_names[2:] # create list of ACS column names (data containing)

# change the data-containing columns to numeric, and replace text with NaN
ACS_df[ACS_column_names_data] = ACS_df[ACS_column_names_data].apply(pd.to_numeric, errors = 'coerce')

ACS_Bay_df = pd.DataFrame(columns = ACS_column_names) # initialize blank dataframe

for ZIP in bay_ZIPs['ZIP_code']:
    temp_df = ACS_df[ACS_df['id'].str.contains(str(ZIP))] # look for ZIP codes from bay_ZIPs in ACS Survey
    ACS_Bay_df = ACS_Bay_df.append(temp_df) # append to blank dataframe
    
ACS_Bay_df = ACS_Bay_df.assign(ZIP_code = np.nan)
ACS_Bay_df['ZIP_code'] = ACS_Bay_df['Geographic Area Name'].str[5:11] # extract ZCTA information as a separate column

The full dataframe from the ACS contains a large amount of information even when filtered just for the ZIP codes of interest, so now filter down to just columns of interest (primary age, sex and race related).

In [12]:
column_selection = ['ZIP_code','Estimate!!SEX AND AGE!!Total population','Estimate!!Total housing units',
                   'Percent Estimate!!SEX AND AGE!!Total population!!Male',
                    'Percent Estimate!!SEX AND AGE!!Total population!!Under 5 years']

ages = [5,10,15,20,25,35,45,55,60,65,75] # start ages from ACS data age ranges

for age in ages:
    if age in [25,35,45,65,75]: # some age ranges are over a decade
        age_range = 'Percent Estimate!!SEX AND AGE!!Total population!!{} to {} years'.format(age, age + 9)
        column_selection.append(age_range)
    elif age in [5,10,15,20,55,60]: # other age ranges are over 5 years
        age_range = 'Percent Estimate!!SEX AND AGE!!Total population!!{} to {} years'.format(age, age + 4)
        column_selection.append(age_range)
    else:
        pass

column_selection.extend(['Percent Estimate!!SEX AND AGE!!Total population!!85 years and over',
                        'Estimate!!SEX AND AGE!!Total population!!Median age (years)',
                        'Percent Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)'])
    
races = ['White','Black or African American','American Indian and Alaska Native','Asian',
         'Native Hawaiian and Other Pacific Islander','Some other race']

for race in races: # iterate over races specified above
    race_name = 'Percent Estimate!!Race alone or in combination with one or more other races!!Total population!!{}'.format(race)
    column_selection.append(race_name)

Finally, rename the columns from their default values (which are too cumbersome) to easily interpretable values

In [13]:
selected_ACS_Bay = ACS_Bay_df[column_selection] # create dataframe based on desired columns

selected_ACS_Bay = selected_ACS_Bay.rename( # rename columns as follows
    columns = {'Estimate!!SEX AND AGE!!Total population':'population',
               'Estimate!!Total housing units':'housing_units',
               'Percent Estimate!!SEX AND AGE!!Total population!!Male':'male_pct',
               'Percent Estimate!!SEX AND AGE!!Total population!!Under 5 years':'under_5_yrs_pct',
               'Percent Estimate!!SEX AND AGE!!Total population!!85 years and over':'85_and_over_pct',
               'Estimate!!SEX AND AGE!!Total population!!Median age (years)':'median_age',
               'Percent Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)':'latino_pct'
              })

for age in ages:
    if age in [25,35,45,65,75]: # some age ranges are over a decade
        selected_ACS_Bay = selected_ACS_Bay.rename(columns = {
            'Percent Estimate!!SEX AND AGE!!Total population!!{} to {} years'.format(age, age + 9):
            '{}_to_{}_yrs_pct'.format(age, age + 9)})
    
    elif age in [5,10,15,20,55,60]: # other age ranges are over 5 years
        selected_ACS_Bay = selected_ACS_Bay.rename(columns = {
            'Percent Estimate!!SEX AND AGE!!Total population!!{} to {} years'.format(age, age + 4):
            '{}_to_{}_yrs_pct'.format(age, age + 4)})

races_labels = ['white','black','native_american','asian','pacific_islander','other'] # abbreviated race names
        
for race, race_label in zip(races,races_labels):
    selected_ACS_Bay = selected_ACS_Bay.rename(columns = {
        'Percent Estimate!!Race alone or in combination with one or more other races!!Total population!!{}'.format(race):
        '{}_pct'.format(race_label)})

selected_ACS_Bay.reset_index(inplace = True, drop = True) # reset index and drop old index from dataframe
selected_ACS_Bay.head()

,ZIP_code,population,housing_units,male_pct,under_5_yrs_pct,5_to_9_yrs_pct,10_to_14_yrs_pct,15_to_19_yrs_pct,20_to_24_yrs_pct,25_to_34_yrs_pct,35_to_44_yrs_pct,45_to_54_yrs_pct,55_to_59_yrs_pct,60_to_64_yrs_pct,65_to_74_yrs_pct,75_to_84_yrs_pct,85_and_over_pct,median_age,latino_pct,white_pct,black_pct,native_american_pct,asian_pct,pacific_islander_pct,other_pct
0,94558,69142,27377,49.9,4.9,5.6,6.9,6.5,5.8,12.3,12.4,13.6,7.1,6.8,10.3,5.3,2.6,41.9,35.3,81.1,1.1,2.3,4.4,0.4,14.8
1,94533,75635,24922,49.6,7.1,7.4,6.8,6.9,7.2,16.1,12.5,13.4,5.5,5.3,6.7,3.6,1.5,34.2,36.3,52.4,19.9,2.1,19.1,2.1,13.6
2,95620,21854,7028,50.9,5.8,7.8,9.0,6.8,8.0,13.5,12.9,12.0,5.9,5.4,6.9,4.8,1.1,34.5,41.2,75.5,3.0,2.1,6.5,1.9,18.8
3,95476,36792,17420,49.9,4.9,4.1,6.0,5.2,4.6,9.3,11.7,12.9,7.1,10.0,12.9,7.7,3.7,48.9,27.1,83.3,1.6,1.1,3.4,0.6,14.5
4,94559,27523,11070,51.1,6.4,5.1,7.0,5.4,6.4,14.7,13.4,13.1,6.4,6.4,9.1,4.4,2.1,38.5,40.1,81.9,1.8,2.1,3.4,0.6,13.8


## Gathering Financial Venue Information from Foursequare

Initialize Foursquare API details

In [14]:
client_ID = '0FDZUBQYVGXXNOOVIZDJVX05AVEJB1ABJSNRV4452VDR42QJ'
client_secret = 'YPTROAKV2NYHPKI0Q5BUFTQNEBXTGYPRKLU3G24ZQBRY4TZK'
version = '20200501' # Foursquare API version

As we only want financial categories of venue, specify them into a dictionary and then create a text string that can be used in the Foursquare API. Note that this will also return venues with a secondary classification as financial, so we will also need to do a second stage of filtering after getting the results from Foursquare. 

In [15]:
location_categories = {'ATM':'52f2ab2ebcbc57f1066b8b56',
                       'bank':'4bf58dd8d48988d10a951735',
                       'check_cashing_service':'52f2ab2ebcbc57f1066b8b2d',
                       'credit_union':'5032850891d4c4b30a586d62',
                       'currency_exchange':'5744ccdfe4b0c0459246b4be',
                       'financial_legal_service':'503287a291d4c4b30a586d65',
                       'insurance_office':'58daa1558bbb0b01f18ec1f1'}

location_categories_list = list(location_categories.values())

location_categories_string = ','.join(location_categories_list)
location_categories_string

'52f2ab2ebcbc57f1066b8b56,4bf58dd8d48988d10a951735,52f2ab2ebcbc57f1066b8b2d,5032850891d4c4b30a586d62,5744ccdfe4b0c0459246b4be,503287a291d4c4b30a586d65,58daa1558bbb0b01f18ec1f1'

In [16]:
SW_corners = []
NE_corners = []

for y in range(precision-1): # iterate through latitudes, (precision - 1) times
    
    for x in range(precision-1): # iterate through longitudes, (precision - 1) times
        SW_corner = ','.join([str(lat_linspace[y]),str(long_linspace[x])]) # foursquare format (latitude, longitude)
        NE_corner = ','.join([str(lat_linspace[y+1]),str(long_linspace[x+1])]) # foursquare format (latitude, longitude)
        SW_corners.append(SW_corner)
        NE_corners.append(NE_corner)

In [17]:
financial_locations = []
url_2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&intent={}&sw={}&ne={}&categoryId={}&limit={}'
    
for i, SW, NE in zip(range(len(SW_corners)),SW_corners,NE_corners):
    
    try:  
        intent = 'browse'
        sw = SW
        ne = NE
        categoryID = location_categories_string
        limit = 5
    
        url_2_formatted = url_2.format(client_ID, client_secret, version, intent, sw, ne, categoryID, limit)
    
        search = requests.get(url_2_formatted).json()['response']['groups'][0]['items']
    
        for v in search:
            try: 
                financial_locations.append((
                    i,
                    v['venue']['id'],
                    v['venue']['name'], 
                    v['venue']['location']['lat'],
                    v['venue']['location']['lng'],
                    v['venue']['location']['postalCode'],
                    v['venue']['categories'][0]['id'],
                    v['venue']['categories'][0]['name']))
        
            except:
                pass
        
    except:
        pass
        
location_col_names = ['square_number','venue_ID','venue_name','latitude','longitude',
                      'ZIP_code','category_ID','category_name']
    
locations = pd.DataFrame(financial_locations, columns = location_col_names)
len(locations)

780

In [18]:
primary_financial = locations[locations['category_ID'].isin(location_categories_list)]
# primary_financial = primary_financial.assign(rating = np.nan)
# primary_financial.set_index('venue_ID', inplace = True)
primary_financial.head()

,square_number,venue_ID,venue_name,latitude,longitude,ZIP_code,category_ID,category_name
0,10,5211b5912fc6de004421fb33,Richard G. Burt,37.331676,-122.221613,95113,503287a291d4c4b30a586d65,Financial or Legal Service
2,15,4b3fbad2f964a5206bad25e3,Wells Fargo - Homestead Crossing,37.337180,-122.066818,95014,4bf58dd8d48988d10a951735,Bank
3,15,51bbd0f7498ea9b7a0c458ea,Edward Jones - Financial Advisor: Todd Bergstrom,37.322694,-122.054243,95014,503287a291d4c4b30a586d65,Financial or Legal Service
4,15,505898e75d24dabaa9bc56dd,Coinstar,37.341700,-122.072300,94024,4bf58dd8d48988d10a951735,Bank
5,15,53a0a24c498ee670915ca8c6,Edward Jones - Financial Advisor: Rafe Ann Du ...,37.349272,-122.050854,94087,503287a291d4c4b30a586d65,Financial or Legal Service


## Visualisation of Number of Results Returned by Foursquare Grid-Search

Create a feature collection that folium can then map, by iterating through the individual sub-divisions and specifying the coordinates with respect to latitude and longitude.

In [19]:
feature_list = []

for y in range(precision-1): # iterate through latitudes, (precision - 1) times
    
    for x in range(precision-1): # iterate through longitudes, (precision - 1) times
        SW_coord = (long_linspace[x],lat_linspace[y])
        NW_coord = (long_linspace[x],lat_linspace[y+1])
        NE_coord = (long_linspace[x+1],lat_linspace[y+1])
        SE_coord = (long_linspace[x+1],lat_linspace[y])
        feature_coords = Feature(geometry = Polygon([[SW_coord,NW_coord,NE_coord,SE_coord,SW_coord]]))
        feature_list.append(feature_coords)
        
feature_collection = FeatureCollection(feature_list)

for i in range((precision - 1) ** 2):
    feature_collection['features'][i]['properties']['square_number'] = i

Summarise the number of samples in each square by referring to the dataframe 'primary_financial'.

In [20]:
samples = pd.DataFrame(columns = ['square_no','count'])

for i in range((precision - 1) ** 2):
    samples.loc[i] = [i,(primary_financial['square_number'] == i).sum()]

Plot a choropleth map with each square showing the number of samples collected from Foursquare. As expected, the greatest density occurs in population centres (e.g. San Francisco, Richmond, Oakland etc.)

In [21]:
latitude = lat_linspace[int((precision - 1) / 2)] # initialise on mid-point latitude
longitude = long_linspace[int((precision - 1) / 2)] # initialise on mid-point longitude

Bay_map = folium.Map(location=[latitude,longitude], tiles = 'OpenStreetMap', zoom_start = 9.5)

folium.Choropleth(
    geo_data = feature_collection,
    data = samples,
    columns = ['square_no','count'],
    key_on = 'feature.properties.square_number',
    fill_color = 'YlGn',
    fill_opacity = 0.7, 
    line_opacity = 0.2).add_to(Bay_map)

# display map
Bay_map

Income tax info from https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2017-zip-code-data-soi

In [22]:
# !curl https://www.irs.gov/pub/irs-soi/17zp05ca.xlsx -o '2017_income_tax.xlsx'

# Results

# Discussion

# Conclusion

In [23]:
# old_code

# rating search

# url_3 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'

# for ID in primary_financial.index:
#     venue_ID = ID
#     url_3_formatted = url_3.format(venue_ID, client_ID, client_secret, version)

#     try:
#         rating_search = requests.get(url_3_formatted).json()['response']['venue']['rating']
#         primary_financial.at[ID,'rating'] = rating_search
        
#     except:
#         pass


# old choropleth

# latitude = 37.77
# longitude = -122.42

# bay_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# bay_map.choropleth(
#     geo_data = geojson_data,
#     data = bay_ZIPs,
#     columns = ['ZIP_code','area'],
#     key_on = 'feature.properties.zip',
#     fill_color = 'YlOrRd',
#     fill_opacity = 0.7, 
#     line_opacity = 0.2)

# # display map
# bay_map

To dos:

- standardize the ACS data (so that it doesn't have magnitude issues)
- collect information on number of bank branches / other financial services per ZIP code
- add in income tax information
- merge together the ZIP codes with the ACS data and foursquare data
- Run clustering or ML analysis on result (+ logistic)

# Sources

https://www.americanbanker.com/news/digital-banking-is-surging-during-the-pandemic-will-it-last

https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index-Datasets.aspx

https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2017-zip-code-data-soi

https://www.udsmapper.org/zcta-crosswalk.cfm

https://data.census.gov/cedsci/table?q=All%20ZCTAs%20in%20United%20States&tid=ACSDP5Y2018.DP05&hidePreview=true&vintage=2018&layer=VT_2018_860_00_PY_D1&cid=DP05_0001E&g=0100000US.860000

https://developer.foursquare.com/docs/build-with-foursquare/categories/


In [24]:
# !curl https://data.sfgov.org/api/geospatial/u5j3-svi6?method=export&format=GeoJSON -o 'bayarea.geojson'